# Notebook for running measurePhotonTransferCurve.py on ComCam data.

Initially written 04 Nov 2019 by Craig Lage, with much help from Andres Plazas.

In [1]:
! eups list -s | grep lsst_distrib

lsst_distrib          18.1.0-2-gb0efc0e+49 	current w_2019_45 setup


In [2]:
import eups
from lsst.daf.persistence import Butler
from lsst.cp.pipe.ptc import MeasurePhotonTransferCurveTask
import sys, os, glob
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf

In [3]:
REPO_DIR = '/project/shared/comCam/'
OUTPUT_DIR = '/home/cslage/ComCam/20191113/'
raftName = 'R22'

In [4]:
# First check the exposure times
filedir = REPO_DIR+'raw/20191113/'
files = np.sort(glob.glob(filedir+'CC_C_20191113_000???/CC_C_20191113_000???_R22_S00.fits'))
numFiles = len(files)
print(numFiles)

for file in files:
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
        
    phdr=hdulist[0].header
    filename = phdr['FILENAME']
    exptime = phdr['EXPTIME']
    imgtype = phdr['IMGTYPE'] 
    print("%s\t%s\t%f"%(filename, imgtype, exptime))

279
CC_C_20191113_000001_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000002_R22_S00.fits	EXPOSE	1.000000
CC_C_20191113_000003_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000004_R22_S00.fits	EXPOSE	1.000000
CC_C_20191113_000005_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000006_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000007_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000008_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000009_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000010_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000011_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000012_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000013_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000014_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000015_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000016_R22_S00.fits	EXPOSE	0.100000
CC_C_20191113_000017_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000018_R22_S00.fits	EXPOSE	0.100000
CC_C_20191113_000019_R22_S00.fits	BIAS	0.000000
CC_C_20191113_000020_R22_S00.fits	EXPOSE	0.200000
CC_C_20191113_000021_R22_S

In [5]:
detector = 0
starting_visit = 2019111300120
ending_visit =   2019111300198
comCam_data = Butler(inputs=REPO_DIR)
md = comCam_data.get('raw', visit=starting_visit, raftName=raftName, detector=detector)
# Below is just a check to make sure we have found the data
print(md.getMetadata()['DATE-OBS'])

2019-11-13T18:58:46.445


In [6]:
# This sets up the pairs.  
pairs = []
visit_1 = starting_visit
while visit_1 < ending_visit+1:
    visit_2 = visit_1 + 2
    pairs.append('%s,%s'%(str(visit_1),str(visit_2)))
    visit_1 += 4 # Skipping the bias frames in between
print(pairs)
print(len(pairs))

['2019111300120,2019111300122', '2019111300124,2019111300126', '2019111300128,2019111300130', '2019111300132,2019111300134', '2019111300136,2019111300138', '2019111300140,2019111300142', '2019111300144,2019111300146', '2019111300148,2019111300150', '2019111300152,2019111300154', '2019111300156,2019111300158', '2019111300160,2019111300162', '2019111300164,2019111300166', '2019111300168,2019111300170', '2019111300172,2019111300174', '2019111300176,2019111300178', '2019111300180,2019111300182', '2019111300184,2019111300186', '2019111300188,2019111300190', '2019111300192,2019111300194', '2019111300196,2019111300198']
20


In [7]:
for detector in range(9):
    args = [REPO_DIR, '--output', OUTPUT_DIR ,'--id', 'raftName=%s'%raftName, 'detector=%d'%detector,  '--visit-pairs']
    for pair in pairs:
        args.append(str(pair))

    args = args + ['--clobber-config', '--clobber-versions','-c', 'makePlots=True', 
                   'ptcFitType=POLYNOMIAL', 'polynomialFitDegree=2', 'maxMeanSignal=100000',
                   '-c', 'isr.doBias=False', 'isr.doDark=False', 'isr.doSuspect=False']

    ptc_struct = MeasurePhotonTransferCurveTask.parseAndRun(args=args)

/opt/lsst/software/stack/stack/miniconda3-4.7.10-4d7b902/Linux64/pex_config/18.1.0-6-gae77429+2/python/lsst/pex/config/config.py:1289: FutureWarning: Config field isr.doAddDistortionModel is deprecated: Camera geometry is incorporated when reading the raw files. This option no longer is used, and will be removed after v19.
  FutureWarning)
/opt/lsst/software/stack/stack/miniconda3-4.7.10-4d7b902/Linux64/pex_config/18.1.0-6-gae77429+2/python/lsst/pex/config/config.py:1289: FutureWarning: Config field isr.doAddDistortionModel is deprecated: Camera geometry is incorporated when reading the raw files. This option no longer is used, and will be removed after v19.
  FutureWarning)
/opt/lsst/software/stack/stack/miniconda3-4.7.10-4d7b902/Linux64/pex_config/18.1.0-6-gae77429+2/python/lsst/pex/config/config.py:1289: FutureWarning: Config field isr.doAddDistortionModel is deprecated: Camera geometry is incorporated when reading the raw files. This option no longer is used, and will be removed af

In [8]:
plt.figure(figsize=(16,8))
plt.subplots_adjust(hspace=0.5)
plt.suptitle("ComCam Gains - 13-Nov-2019", fontsize=24)
plotcounter = 0
for detector in range(9):
    plotcounter += 1
    plt.subplot(3,3,plotcounter)
    plt.title("Detector%d"%detector)
    gain_pickle_file = OUTPUT_DIR+'calibrations/ptc/ptcDataGainAndNoise-det%03d.pkl'%detector
    gain_file = open(gain_pickle_file, 'rb')
    gain_data = pkl.load(gain_file)
    amps = gain_data['gain'].keys()
    gains = []
    gain_err = []
    names = []
    for amp in amps:
        gains.append(gain_data['gain'][amp][0])
        gain_err.append(gain_data['gain'][amp][1])
        names.append(amp)
    plt.errorbar(amps, gains, yerr=gain_err, marker = 'x')
    plt.ylim(0.0, 1.5)
    plt.xticks(list(range(16)),names, fontsize=8)
    
plt.savefig(OUTPUT_DIR+'plots/Gain_Summary_13Nov19.pdf')
